# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
print(os.getenv('HF_ENDPOINT'))
os.environ["HF_HOME"] = "/root/autodl-tmp/cache"
print(os.getenv('HF_HOME'))

https://hf-mirror.com
/root/autodl-tmp/cache


In [2]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [3]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/root/miniconda3/envs/jike/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/envs/jike/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/root/miniconda3/envs/jike/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [8]:
input_text = "解释下师卦是什么？"

In [9]:
response, history = base_model.chat(tokenizer, query=input_text)
print(response)

师卦是《易经》中的第八卦，由两个阴爻夹一个阳爻构成，象征着军队或力量。师卦的含义广泛，可以表示军队、力量、领导、教导等。在《易经》的八卦中，阳爻代表阳刚之气，阴爻代表阴柔之气。阳爻在阴爻之中，象征着军队或力量 contained within，阴爻则代表军队或力量的外部表现。

师卦的卦辞描述了军队或力量的特点，如“天行健，君子以自强不息”。这句话意味着，像天行健（天行刚健）一样，君子应该自强不息，不断努力。师卦鼓励人们积极向上，不断自我提高，以达到成功。

在师卦中，阳爻代表领导或教导，阴爻代表士兵或追随者。领导应该以阳刚之气为基础，阳爻代表阳刚之气，阴爻则代表阴柔之气。领导应该在士兵和追随者之中发挥阳刚之气，以激励他们，带领他们前进。

总之，师卦象征着力量、领导和教育，鼓励人们积极向上，自强不息。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是《易经》中的第五卦，由两个阳爻夹一个阴爻构成。这个卦象象征着诉讼、争端和 conflict。

讼卦的意义在于教导我们如何应对和解决争端和冲突。它告诉我们，在面对矛盾和争议时，应该以和平、公正和理性的态度来解决问题，而不是通过诉讼等激烈的方式来解决。

讼卦建议我们在处理争端和冲突时，应该采取积极的和平方式，尽量通过对话、协商和妥协来解决问题。同时，我们也应该保持冷静和理智，避免情绪化的决策和行动。

此外，讼卦还告诉我们，解决争端和冲突需要时间和耐心。我们应该保持耐心，不要急于求成，应该通过深入的沟通和协商，找到一个公正合理的解决方案。

总之，讼卦教导我们在面对争端和冲突时，应该以和平、公正和理性的态度来解决问题，而不是通过诉讼等激烈的方式来解决。我们应该保持冷静和理智，避免情绪化的决策和行动，并保持耐心，通过深入的沟通和协商来找到一个公正合理的解决方案。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [12]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20250811_221503"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"
print(peft_model_path)

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

models/THUDM/chatglm3-6b-epoch3-20250811_221503


In [10]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

In [14]:
base_response, base_history = base_model.chat(tokenizer, query="解释下师卦是什么？")
print(base_response)

师卦，象征着军队与集体力量，通过上下卦的组合：上卦为坤（地），下卦为坎（水），形成“地中有水”的独特卦象，寓意君子应如大地般容纳和培育大众。此卦的出现暗示问卦者将在没有灾祸与困扰的情况下，处理与军队和集体相关的事务。

《象辞》指出，师卦代表着吉祥与无咎，其关键在于选择品德高尚、受到尊重的长者作为领导者。而在工作与生活当中，面对重重困难与挑战，师卦鼓励人们应以包容之心对待他人，勤勉努力来克服障碍。务必行事公正无私，以化解潜在的危机与风险。

在运势方面，师卦显得尤为谨慎，需避免独断专行，倡导与他人密切合作，以确保稳妥应对竞争和阻力。经商者在此卦的影响下，既可利用现有积蓄进行大的市场拓展，又需保持警惕，与他人良好沟通，以化险为夷。

对于求名与婚恋，师卦提醒要注重专注与慎重，小心避免纠纷与误会。决策方面，此卦则强调应以灵活性与坚韧意志迎接挑战，追求事业成功的同时，保持老成持重的态度，以确保最终的成就。


### 微调前后效果对比

In [12]:
base_response, ft_response = compare_chatglm_results("解释下师卦是什么？", base_model, qlora_model, training_tag)

问题：解释下师卦是什么？

原始输出：
师卦，象征着军队与集体力量，通过上下卦的组合：上卦为坤（地），下卦为坎（水），形成“地中有水”的独特卦象，寓意君子应如大地般容纳和培育大众。此卦的出现暗示问卦者将在没有灾祸与困扰的情况下，处理与军队和集体相关的事务。

《象辞》指出，师卦代表着吉祥与无咎，其关键在于选择品德高尚、受到尊重的长者作为领导者。而在工作与生活当中，面对重重困难与挑战，师卦鼓励人们应以包容之心对待他人，勤勉努力来克服障碍。务必行事公正无私，以化解潜在的危机与风险。

在运势方面，师卦显得尤为谨慎，需避免独断专行，倡导与他人密切合作，以确保稳妥应对竞争和阻力。经商者在此卦的影响下，既可利用现有积蓄进行大的市场拓展，又需保持警惕，与他人良好沟通，以化险为夷。

对于求名与婚恋，师卦提醒要注重专注与慎重，小心避免纠纷与误会。决策方面，此卦则强调应以灵活性与坚韧意志迎接挑战，追求事业成功的同时，保持老成持重的态度，以确保最终的成就。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250811_221503）：
[gMASK]sop 解释下师卦是什么？ 师卦，象征着军队与集体力量，通过上下卦的组合：上卦为坤（地），下卦为坎（水），形成“地中有水”的独特卦象，寓意君子应如大地般容纳和培育大众。此卦的出现暗示问卦者将在没有灾祸与困扰的情况下，处理与军队和集体相关的事务。

《象辞》指出，师卦代表着吉祥与无咎，其关键在于选择品德高尚、受到尊重的长者作为领导者。而在工作与生活当中，面对重重困难与挑战，师卦鼓励人们应以包容之心对待他人，勤勉努力来克服障碍。务必行事公正无私，以化解潜在的危机与风险。

在运势方面，师卦显得尤为谨慎，需避免独断专行，倡导与他人密切合作，以确保稳妥应对竞争和阻力。经商者在此卦的影响下，既可利用现有积蓄进行大的市场拓展，又需保持警惕，与他人良好沟通，以化险为夷。

对于求名与婚恋，师卦提醒要注重专注与慎重，小心避免纠纷与误会。决策方面，此卦则强调应以灵活性与坚韧意志迎接挑战，追求事业成功的同时，保持老成持重的态度，以确保最终的成就。


In [16]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是周易中的一个重要卦象，象征着争讼与对立。该卦由两个卦象构成：上卦为坎，表示水；下卦为宣，表示谋略。根据《象辞》的解读，讼卦寓意着争讼之事，必须通过合法的途径来解决。

从哲学角度看，讼卦警告我们在处理争讼问题时要谨慎，必须通过合法的途径来解决，以确保自身利益不受损害。这表明，在面临矛盾和争议时，我们应该遵守法律法规，尊重权威，以维护公平和正义。

在运势方面， stratified.com的数据显示，讼卦显示了谋略和智慧，提醒我们在此领域要小心谨慎，避免与权威产生冲突。根据运势解析，此卦显示了在事业和财务上的不安定，因此，我们需要在此方面加强努力，以实现稳定的发展。

在婚恋方面，讼卦提醒我们要尊重对方的意愿，避免争讼，以维护幸福的婚姻生活。而在财运方面，该网站显示，我们需要遵守法律法规，以避免财运受损。

总结起来，讼卦强调谋略和智慧，提醒我们要通过合法的途径来解决争讼，以确保自身利益不受损害。在此过程中，我们需要保持冷静，以避免与权威产生冲突，同时要尊重法律法规，以确保财务和事业上的稳定。在婚恋方面，我们需要尊重对方的意愿，以维护幸福的婚姻生活。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250811_221503）：
[gMASK]sop 周易中的讼卦是什么含义 在周易中，讼卦象征着争讼与对立，其卦象为上卦乾（天）与下卦坎（水）相叠，二者的流向背道而驰，表明事理的乖舛。虽然吞并和获利的机会存在，但必须警惕潜在的危险与风险，尤其在中间阶段虽有吉利，最终结果可能却是凶险。因此，得此卦者应谨慎对待事物，防止口舌之争。

根据《象辞》的解读，君子应从此卦中吸取教训，避免争讼，尤其在谋划事情之初需审慎对待，不轻言立场。北宋易学家邵雍指出此卦预示身心的不安和与他人多有争议，必须做到修身养性、谨慎行事。

从运势的角度看，此卦显示出事与愿违的趋势，凡事不顺，易遭小人加害，因此应设法防范潜在的陷阱。在事业上初起顺利虽有利益，但随之而来的挫折促使人们警惕、避开诉讼矛盾。在经商方面，坚持和气生财、追求公正是获得好结果的关键。

因此，综上所述，讼卦教导我们在面临争讼和竞争时，需尽量保持冷静，并以和睦的态度解决矛盾，才能避免不必要的损失与灾祸。


In [25]:
base_response, ft_response = compare_chatglm_results("解释师卦是什么？", base_model, qlora_model, training_tag)

问题：解释师卦是什么？

原始输出：
师卦，象征着军队与集体力量，通过上下卦的组合：上卦为坤（地），下卦为坎（水），形成“地中有水”的独特卦象，寓意君子应如大地般容纳和培育大众。此卦的出现暗示问卦者将在没有灾祸与困扰的情况下，处理与军队和集体相关的事务。

《象辞》指出，师卦代表着吉祥与无咎，其关键在于选择品德高尚、受到尊重的长者作为领导者。而在工作与生活当中，面对重重困难与挑战，师卦鼓励人们应以包容之心对待他人，勤勉努力来克服障碍。务必行事公正无私，以化解潜在的危机与风险。

在运势方面，师卦显得尤为谨慎，需避免独断专行，倡导与他人密切合作，以确保稳妥应对竞争和阻力。经商者在此卦的影响下，既可利用现有积蓄进行大的市场拓展，又需保持警惕，与他人良好沟通，以化险为夷。

对于求名与婚恋，师卦提醒要注重专注与慎重，小心避免纠纷与误会。决策方面，此卦则强调应以灵活性与坚韧意志迎接挑战，追求事业成功的同时，保持老成持重的态度，以确保最终的成就。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250811_221503）：
[gMASK]sop 解释师卦是什么？ 师卦，象征着军队与集体力量，通过上下卦的组合：上卦为坤（地），下卦为坎（水），形成“地中有水”的独特卦象，寓意君子应如大地般容纳和培育大众。此卦的出现暗示问卦者将在没有灾祸与困扰的情况下，处理与军队和集体相关的事务。

《象辞》指出，师卦代表着吉祥与无咎，其关键在于选择品德高尚、受到尊重的长者作为领导者。而在工作与生活当中，面对重重困难与挑战，师卦鼓励人们应以包容之心对待他人，勤勉努力来克服障碍。务必行事公正无私，以化解潜在的危机与风险。

在运势方面，师卦显得尤为谨慎，需避免独断专行，倡导与他人密切合作，以确保稳妥应对竞争和阻力。经商者在此卦的影响下，既可利用现有积蓄进行大的市场拓展，又需保持警惕，与他人良好沟通，以化险为夷。

对于求名与婚恋，师卦提醒要注重专注与慎重，小心避免纠纷与误会。决策方面，此卦则强调应以灵活性与坚韧意志迎接挑战，追求事业成功的同时，保持老成持重的态度，以确保最终的成就。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [18]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [19]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是《周易》中的第一卦，代表着天，象征着动，以及刚强。卦辞中提到的'天行健，君子以自强不息'，提示着在这卦象下，君子应当积极进取，坚韧不拔，才能达到完美的境界。乾卦的核心哲学在于强调'刚强'和'希望'，代表着天性强健，刚毅果敢的特性，象征着成功和发展的开始。

乾卦由两个部分组成，上卦是红色，下卦是青色。上卦的六条线全部由阳爻（表示刚强）组成，象征着太阳照耀，下卦的六条线中有两条阳爻，四条阴爻，象征着月亮照耀。这种组合象征着天与地之间的交互，以及君子们既要刚强，又要谨慎行事，才能在事业上取得成功。

乾卦所象征的刚强和坚定，代表着太阳照耀，象征着力量和信心，以及天与地相辅相成的关系。它提醒我们，在事业上要坚定信念，积极进取，以坚韧不拔的精神追求成功。同时，在处理事务时，要稳重冷静，善于观察，才能达到理想的结果。

在婚恋方面，乾卦象征着男方的刚强和坚定，以及女方的小鸟依人。它提示我们要认真对待婚恋，注重互相了解，尊重对方，才能实现美满的婚姻。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 在《周易》中，乾卦是六十四卦之首，以六个阳爻构成，象征天，意指大吉大利，形象表明了自强不息的精神。卦辞中提到的'元、亨、利、贞'四字，代表着成功的基础、通达的道路、带来利益的力量以及持守正道的品格。乾卦的核心哲学在于强调刚健和主动向上的力量，君子应以此卦象为法则，持续进行自我提升，不断进取，才可持久通畅。

乾卦所象征的天道运行，意指天行健；于是，君子应效仿天道之刚强，坚持不懈，追求卓越。在事业上，此卦象征着名利双收，良好的时机正等着把握，但同时也提醒着要保持冷静，避免因过于强势而带来问题。在经营方面，乾卦指明了良好的利润空间，但也警示不可急功近利。

在婚恋方面，尽管阳光明媚，阴柔气息相对减少，但如果能够柔和并互补，必能形成美好的结合。此外，决策方面也不乏成功的机会，只需坚持正直与公正的原则，积累智慧和品德，方可克服困难，实现理想。


In [16]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦是易经中的一卦，代表的是坤卦，它的卦象是由两个坤卦叠加而成。在卜筮时，这个卦象象征着地中的水（代表阴柔），以及 associated with it 的一些事物（代表阳刚）。地水师卦的卦象表示了地的承载能力，象征着大地包容万物，具有强大的包容力。

在哲学层面上，地水师卦象征着阳刚生于阴柔之中，预示着事物的发展将有著新的动向。在卜筮时，这种卦象被认为具有吉祥的意义，预示着将会面临新的挑战，但也能带来成功。

地水师卦的运势特点如下：

- 事业：面临新的挑战，需要积极应对，寻求新的发展机会。
- 爱情：感情可能受到 external 因素的影响，需要多沟通，理解对方。
- 财运：事业顺利，能够得到财富。
- 健康：身体可能会有状况，需要保持健康的生活方式。

地水师卦的经商运势：

- 事业：需要积极拓展业务，提高自身竞争力，寻求新的合作伙伴。
- 投资：不宜于进行高风险投资，需要谨慎对待。
- 经商：适合以稳健为主，不宜冒险，需要仔细考虑。

总结起来，地水师卦的总体运势较为顺利，但也需要注意 new challenges and opportunities, and the need for careful consideration and understanding.


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由坎卦（水）和坤卦（地）组成的卦象，代表地下的水，象征润泽大地的力量。根据《易经》中的解释，地水师卦预示着吉祥如意，具有强大的力量，可以克服各种困难。

根据传统解卦，地水师卦分为两个卦象，第一个卦象是坎卦（水），代表灵活变化的力量，具有吉祥如意，靠智慧取悦于人；第二个卦象是坤卦（地），代表刚健柔顺，具有强大的力量，可以克服各种困难。

地水师卦的核心理念是：吉祥如意，靠智慧取悦于人，靠刚健柔顺的力量，可以克服各种困难。


In [17]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是周易中的一卦，代表诉讼的意义。卦名“讼”由上卦坎（水）和下卦乾（天）组成，上卦为云，下卦为天。代表险恶的刚直之象。讼卦的核心哲学是：天（乾）刚直，云（坎）险恶，刚柔相济，可以免患失。

讼卦的核心哲学是：刚直刚硬，不能容悦于人，因此必致刚柔相济。刚可以济柔，刚又能柔，则无失矣。

讼卦的取裁：

刚直刚硬，不能容悦于人，因此必致刚直硬板。

刚可以济柔，刚又能柔，则无失矣。

刚直硬板，可以容悦于人，则无失矣。

诉词刚硬，必致刚硬相济。

刚直硬板，必须得人相助。

source: 《易经》


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么卦象

 讼卦是周易卦象中的一卦，由上卦坎（水）和下卦乾（天）组成，象征着天神下降于坎水之中，具有刚健刚毅之象。讼卦的卦象具有云雾聚集、天光破云的意象，象征着事情将会顺利发展，宜早做准备。

讼卦的时运是：
 初爻：阳爻（天）刚健刚毅
 二爻：阳爻（天）刚健刚毅
 三爻：阳爻（天）刚健刚毅
 四爻：阳爻（天）刚健刚毅
 初爻：阳爻（天）刚健刚毅
 限运：
 阳爻（天）刚健刚毅
 阴爻（天）刚柔温和
 阳爻（天）刚健刚毅
 阳爻（天）刚健刚毅

讼卦的解卦是：
 初爻：阳爻（天）刚健刚毅
 二爻：阳爻（天）刚健刚毅
 三爻：阳爻（天）刚健刚毅
 初爻：阳爻（天）刚健刚毅
 限运：
 阳爻（天）刚健刚毅
 阴爻（天）刚柔温和
 阳爻（天）刚健刚毅
 阳爻（天）刚健刚毅

讼卦的卦象和时运都预示着刚健刚毅的特性，天神下降于坎水之中，具有刚健刚毅之象。在事业和生活中遇到困难和挑战时，需要积极应对，宜早做准备，不宜冒险，切勿冒进。在处理问题和决策时，要冷静分析，不宜草率行事，宜等待时机的到来。同时，要注重刚健刚毅的态度，积极应对各种困难和挑战，从而获得成功。


#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [18]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [19]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》中的第一卦，卦象由六个阳爻（即阳性的木）排列而成，它们从下至上排列，看起来像是天上的云彩，因此得名“乾”。乾卦象征着刚强、坚毅和独立，以及与天地之间的联系。

乾卦的卦辞是：“乾为天，为父，为君，亲阳人。亲，先祖，家远的代际，宜先祖；国，先王，宜治国；夫，配偶，宜婚配；父，尊亲，宜尊父；君，朋友，宜友道。”卦辞描述了乾卦的内涵，它既象征着天，也象征着人。作为天，它与“先祖”和“先王”有关，表示宇宙的起点和万物的基础。作为人，它与“父”和“君”有关，表示人类的社会性和文化传承。

在《易经》中，乾卦是“八卦”之一，它既象征着阳，也象征着“刚”和“强”。乾卦的卦象既象征着“天”，也象征着“人”，它是“八卦”中最为重要的卦之一，表示宇宙的运行和万物的生成。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？乾卦是《易经》中的第一卦，也是六十四卦中的首卦，它的卦象是由两个阳爻夹一个阴爻构成，象征着天，也象征着刚强和坚毅。乾卦的卦辞是“元，亨，利，贞”，意味着“ beginning, success, profit, integrity”。它表示万事万物的本源和原理，是成功的基础。乾卦 also means "the heaven", and it is a symbol of the universe, with six lines of yin and one line of yang. It represents the power and strength of the world, and is a symbol of integrity.


In [20]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
师卦（师的卦象为乾为天，地为地）主动向被动，主动而被动，地震师，吉。象征师兵，兵者国之大事，故放师卦于和创新，守师卦则不进。

师卦是由天（乾）和地（坤）组成的卦象，乾为天，坤为地。师卦是《易经》中第八卦，属于坤卦（ earth）系列卦象，坤卦下卦为坤，上卦为乾，乾为天，坤为地。

师卦象征着师兵，主动向被动，主动而被动，地震师，吉。象征师兵，兵者国之大事，故放师卦于和创新，守师卦则不进。

师卦最佳状态为师的（师卦上卦为乾，下卦为坤），乾为天，坤为地，师卦象征师兵，主动向被动，主动而被动，地震师，吉。象征师兵，兵者国之大事，故放师卦于和创新，守师卦则不进。

在占卦中，师卦卦象为吉凶相生，以阳刚为上，以顺为正，symbolism of the earth and sky，象征天地。师卦：师 师 师。三个师字，下坤上乾。

师卦卦象分析：师卦是乾卦下坤卦上，乾为天，坤为地，象征天和地。师卦象征师兵，主动向被动，主动而被动，地震师，吉。

师卦卦象占卜：占师卦，得师卦，吉。

师卦卦象应用：师卦象征师兵，主动向被动，主动而被动，象征天和地。适用于天和地的事物，占师卦 Creature and earth。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦 师卦原文：师。贞，丈人吉，无咎。象曰：地中有水，师。君子以容民畜众。白话文解释：师卦象征军队指挥，无灾祸。《象辞》说：下卦为坎（水），上卦为坤（地），如大地容纳江河，君子应容纳众人。《断易天机》解：师卦坤上坎下，象征军众，需德高长者统率以吉无咎。北宋易学家邵雍解：忧劳动众，公正无私排难。得卦者应包容他人，努力排除困难。台湾国学大儒傅佩荣解：时运包容他人，财运有财需珍惜，家宅旧亲联姻吉，身体腹胀调气。传统解卦：异卦（下坎上坤），“师”指军队。坎为水险，坤为地顺，寓兵于农，用兵应顺势，故化凶为吉。


In [21]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
讼卦（卦名：讼）是《易经》中的第八卦，属于雷（卦头）坎（卦体）离（卦象）两种卦象的组合。讼卦象征诉讼、争议、争端等诉讼方面的现象。卦形为雷上坎下，雷象征刚毅，坎象征险，离象征烈。卦象下卦为坎，上卦为离，险险相间，刚毅相结合。

讼卦的卦辞：“讼，吉凶交战，得利藏损。”卦象下坎上离，坎险离光，光争下，险林光，刚毅争利，光损藏吉。

蔽卦（卦名：蔽）是《易经》中的第六卦，属于离（卦头）坤（卦体）震（卦象）两种卦象的组合。蔽卦象征屏蔽、遮蔽，阻止邪恶势力，弘扬正道。

蔽卦的卦辞：“蔽，利ulo，吉凶凶险皆屏蔽。”卦象下离上坤，离火遮蔽，坤地顺从，刚毅相合，和顺利害，吉祥安全。

 src="http://www.qk6.com/article/html/277776.html" target="_blank">http://www.qk6.com/article/html/277776.html


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 天水讼卦 讼卦原文：讼。有孚，窒惕，中吉，终凶。利见大人，不利涉大川。象曰：天与水违行，讼。君子以做事谋始。白话文解释：讼卦象征虽有利可图但需警惕。事情初吉后凶，利于见贵人，不宜涉水。《象辞》说：上卦为乾（天），下卦为坎（水），天水相隔，事理不合，君子需慎重谋事。《断易天机》解：讼卦乾上坎下，刚遇险，必有争论，多不吉。北宋易学家邵雍解：天高水深，远离不亲，慎谋退守则无凶。得此卦者，身心不安，多争诉，宜修身养性。台湾国学大儒傅佩荣解：时运受阻，财运初谨慎终获利，家宅君子求淑女，身体预防胜于治疗。传统解卦：异卦（下坎上乾），刚健遇险，彼此反对，生争讼，需慎重戒惧。
